In [1]:
import os
import polars as pl

# Import Dataset

In [2]:
local_path='dataset/'
file_country_codes='country_codes.csv'
file_gdp_data='gdp_data.csv'
auto='auto'

In [3]:
# get current directory
path = os.getcwd()
print("Current Directory", path)
print()
 
# parent directory
parent = os.path.dirname(path)
print("Parent directory", parent)
print()

# country code dataset path
dataset_path_country_code=os.path.join(parent,local_path,file_country_codes)
print("Dataset Country Code path",dataset_path_country_code)
print()

# gdp dataset path
dataset_path_gdp_data=os.path.join(parent,local_path,file_gdp_data)
print("Dataset GDP path",dataset_path_gdp_data)
print()

# auto path
path_auto=os.path.join(parent,auto)
print("Dump path",path_auto)

Current Directory /Users/baptistebeaurain/Google Drive/learning/Python/polars_explo/project/gdp/notebook

Parent directory /Users/baptistebeaurain/Google Drive/learning/Python/polars_explo/project/gdp

Dataset Country Code path /Users/baptistebeaurain/Google Drive/learning/Python/polars_explo/project/gdp/dataset/country_codes.csv

Dataset GDP path /Users/baptistebeaurain/Google Drive/learning/Python/polars_explo/project/gdp/dataset/gdp_data.csv

Dump path /Users/baptistebeaurain/Google Drive/learning/Python/polars_explo/project/gdp/auto


In [4]:
df_country_code=pl.read_csv(dataset_path_country_code)

print("shape:", df_country_code.shape)

df_country_code.head(5)

shape: (250, 2)


Code,Name
str,str
"""TWN""","""Taiwan"""
"""AFG""","""Afghanistan"""
"""ALB""","""Albania"""
"""DZA""","""Algeria"""
"""ASM""","""American Samoa…"


In [5]:
df_gdp_data=pl.read_csv(dataset_path_gdp_data)
df_gdp_data=df_gdp_data.sort(['Country Code','Year'])

col_rename={
    'Country Name':'cty_name',
    'Country Code':'cty_code',
    'Year': 'year',
    'Value': 'gdp'
    }

df_gdp_data = df_gdp_data.rename(col_rename)

print("shape:", df_gdp_data.shape)

df_gdp_data.head(5)

shape: (11507, 4)


cty_name,cty_code,year,gdp
str,str,i64,f64
"""Aruba""","""ABW""",1994,1.3302e9
"""Aruba""","""ABW""",1995,1.3207e9
"""Aruba""","""ABW""",1996,1.3799e9
"""Aruba""","""ABW""",1997,1.5318e9
"""Aruba""","""ABW""",1998,1.6654e9


# Challenges

 ## Calculate GDP evolution per year

In [6]:
gdp_df = df_gdp_data.sort(['cty_code', 'year']).select([
    pl.col('*'),
    pl.col('gdp').shift(1).over('cty_code').alias('prev_gdp'),
    pl.col('year').shift(1).over('cty_code').alias('prev_year'),
])

gdp_evol_df=gdp_df.with_columns(
    (((pl.col('gdp')/pl.col('prev_gdp')))).alias('gdp_evol'),
    (pl.col('year')-pl.col('prev_year')).alias('nb_year'),
)
gdp_evol_df=gdp_evol_df.with_columns(
    ((pl.col('gdp_evol').pow(1/pl.col('nb_year'))-1)*100).alias('gdp_evol_percent'),
)

gdp_evol_df = gdp_evol_df.filter((pl.col('prev_year').is_not_null()))

# Display or save the result
print(gdp_evol_df)
# OR
gdp_evol_df.write_csv(os.path.join(path_auto,'gdp_evol_df.csv'))


shape: (11_251, 9)
┌──────────┬──────────┬──────┬───────────┬───┬───────────┬──────────┬─────────┬──────────────────┐
│ cty_name ┆ cty_code ┆ year ┆ gdp       ┆ … ┆ prev_year ┆ gdp_evol ┆ nb_year ┆ gdp_evol_percent │
│ ---      ┆ ---      ┆ ---  ┆ ---       ┆   ┆ ---       ┆ ---      ┆ ---     ┆ ---              │
│ str      ┆ str      ┆ i64  ┆ f64       ┆   ┆ i64       ┆ f64      ┆ i64     ┆ f64              │
╞══════════╪══════════╪══════╪═══════════╪═══╪═══════════╪══════════╪═════════╪══════════════════╡
│ Aruba    ┆ ABW      ┆ 1995 ┆ 1.3207e9  ┆ … ┆ 1994      ┆ 0.99286  ┆ 1       ┆ -0.713986        │
│ Aruba    ┆ ABW      ┆ 1996 ┆ 1.3799e9  ┆ … ┆ 1995      ┆ 1.044839 ┆ 1       ┆ 4.483926         │
│ Aruba    ┆ ABW      ┆ 1997 ┆ 1.5318e9  ┆ … ┆ 1996      ┆ 1.110121 ┆ 1       ┆ 11.012146        │
│ Aruba    ┆ ABW      ┆ 1998 ┆ 1.6654e9  ┆ … ┆ 1997      ┆ 1.087163 ┆ 1       ┆ 8.716265         │
│ …        ┆ …        ┆ …    ┆ …         ┆ … ┆ …         ┆ …        ┆ …       ┆ …         

In [7]:
# Display statistics for numeric columns
stats = gdp_evol_df.describe()
print(stats)

shape: (9, 10)
┌────────────┬────────────┬──────────┬───────────┬───┬───────────┬──────────┬──────────┬───────────┐
│ describe   ┆ cty_name   ┆ cty_code ┆ year      ┆ … ┆ prev_year ┆ gdp_evol ┆ nb_year  ┆ gdp_evol_ │
│ ---        ┆ ---        ┆ ---      ┆ ---       ┆   ┆ ---       ┆ ---      ┆ ---      ┆ percent   │
│ str        ┆ str        ┆ str      ┆ f64       ┆   ┆ f64       ┆ f64      ┆ f64      ┆ ---       │
│            ┆            ┆          ┆           ┆   ┆           ┆          ┆          ┆ f64       │
╞════════════╪════════════╪══════════╪═══════════╪═══╪═══════════╪══════════╪══════════╪═══════════╡
│ count      ┆ 11251      ┆ 11251    ┆ 11251.0   ┆ … ┆ 11251.0   ┆ 11251.0  ┆ 11251.0  ┆ 11251.0   │
│ null_count ┆ 0          ┆ 0        ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0      ┆ 0.0      ┆ 0.0       │
│ mean       ┆ null       ┆ null     ┆ 1991.7189 ┆ … ┆ 1990.7109 ┆ 1.085501 ┆ 1.007999 ┆ 8.462492  │
│            ┆            ┆          ┆ 58        ┆   ┆ 59        ┆          

# Descriptive Stats

## Top 10 GDP Increase

In [8]:
top_5_values = gdp_evol_df.sort('gdp_evol_percent', descending=True)

top_5_values.head(10)

cty_name,cty_code,year,gdp,prev_gdp,prev_year,gdp_evol,nb_year,gdp_evol_percent
str,str,i64,f64,f64,i64,f64,i64,f64
"""Equatorial Gui…","""GNQ""",1965,6.4748e7,1.2712e7,1964,5.093292,1,409.329235
"""Congo, Dem. Re…","""COD""",2000,1.9088e10,4.7113e9,1999,4.05158,1,305.158039
"""Oman""","""OMN""",1974,1.6459e9,4.8303e8,1973,3.407458,1,240.745788
"""Saudi Arabia""","""SAU""",1974,4.5413e10,1.4947e10,1973,3.038186,1,203.818622
"""Qatar""","""QAT""",1974,2.4014e9,7.9388e8,1973,3.024878,1,202.487783
"""Iraq""","""IRQ""",1990,1.7989e11,6.5641e10,1989,2.740434,1,174.043385
"""Kiribati""","""KIR""",1974,8.5637e7,3.1711e7,1973,2.70058,1,170.05802
"""Brunei Darussa…","""BRN""",1974,1.0736e9,4.3309e8,1973,2.478866,1,147.88661
"""Kuwait""","""KWT""",1974,1.3005e10,5.4083e9,1973,2.404598,1,140.45983


# Top 10 GDP decrease

In [9]:
bottom_5_values = gdp_evol_df.sort('gdp_evol_percent', descending=False)

bottom_5_values.head(10)

cty_name,cty_code,year,gdp,prev_gdp,prev_year,gdp_evol,nb_year,gdp_evol_percent
str,str,i64,f64,f64,i64,f64,i64,f64
"""Vietnam""","""VNM""",1989,6.2933e9,2.5424e10,1988,0.247536,1,-75.246415
"""Serbia""","""SRB""",2000,6.5402e9,1.8409e10,1999,0.355267,1,-64.473259
"""Argentina""","""ARG""",2002,9.7724e10,2.6870e11,2001,0.363696,1,-63.630374
"""Rwanda""","""RWA""",1994,7.5364e8,1.9715e9,1993,0.38226,1,-61.773957
"""Congo, Dem. Re…","""COD""",2001,7.4382e9,1.9088e10,2000,0.389678,1,-61.032214
"""Indonesia""","""IDN""",1998,1.0162e11,2.2971e11,1997,0.442392,1,-55.760839
"""Mozambique""","""MOZ""",1987,2.3541e9,5.2472e9,1986,0.448643,1,-55.135684
"""Congo, Dem. Re…","""COD""",1964,2.8815e9,6.2132e9,1963,0.463779,1,-53.622097
"""Libya""","""LBY""",2011,3.4699e10,7.4773e10,2010,0.46406,1,-53.593959


In [17]:
recession_df = gdp_evol_df.filter(
    (pl.col('gdp_evol_percent')<0) & (pl.col('year')==2013))

print(recession_df.shape)

recession_df

(32, 9)


cty_name,cty_code,year,gdp,prev_gdp,prev_year,gdp_evol,nb_year,gdp_evol_percent
str,str,i64,f64,f64,i64,f64,i64,f64
"""Afghanistan""","""AFG""",2013,2.0046e10,2.0537e10,2012,0.97613,1,-2.387006
"""American Samoa…","""ASM""",2013,6.41e8,6.44e8,2012,0.995342,1,-0.465839
"""Antigua and Ba…","""ATG""",2013,1.1929e9,1.2114e9,2012,0.98474,1,-1.526013
"""Bahamas, The""","""BHS""",2013,1.0677e10,1.0720e10,2012,0.995961,1,-0.403899
"""Barbados""","""BRB""",2013,4.6125e9,4.6564e9,2012,0.990583,1,-0.941725
"""Brunei Darussa…","""BRN""",2013,1.8094e10,1.9048e10,2012,0.949882,1,-5.011764
"""Bhutan""","""BTN""",2013,1.7983e9,1.8237e9,2012,0.986095,1,-1.390497
"""Central Africa…","""CAF""",2013,1.5186e9,2.1842e9,2012,0.695255,1,-30.474475
"""Cyprus""","""CYP""",2013,2.4085e10,2.5041e10,2012,0.961791,1,-3.820876
